<a href="https://colab.research.google.com/github/arkadijshmelev/whether_to_take_an_umbrella/blob/master/4_%D0%A1%D0%BB%D1%83%D1%87%D0%B0%D0%B9%D0%BD%D1%8B%D0%B9_%D0%BB%D0%B5%D1%81_%D0%BF%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D0%BA%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn import tree
from sklearn import ensemble
from sklearn import metrics
from sklearn.model_selection import GridSearchCV

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
# Загрузим данные с google drive.
data = drive.CreateFile({'id': '1dyshTwrVTBuIzjeC575Ojg--DlkAhstv'})
data.GetContentFile('data/weatherAUS.csv')

In [ ]:
# Загружаем файл.
df = pd.read_csv('data/weatherAUS.csv')
# Выведем для просмотра.
display(df.head())
# Выведем информацию о датафрейме.
df.info()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145460 entries, 0 to 145459
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           145460 non-null  object 
 1   Location       145460 non-null  object 
 2   MinTemp        143975 non-null  float64
 3   MaxTemp        144199 non-null  float64
 4   Rainfall       142199 non-null  float64
 5   Evaporation    82670 non-null   float64
 6   Sunshine       75625 non-null   float64
 7   WindGustDir    135134 non-null  object 
 8   WindGustSpeed  135197 non-null  float64
 9   WindDir9am     134894 non-null  object 
 10  WindDir3pm     141232 non-null  object 
 11  WindSpeed9am   143693 non-null  float64
 12  WindSpeed3pm   142398 non-null  float64
 13  Humidity9am    142806 non-null  float64
 14  Humidity3pm    140953 non-null  float64
 15  Pressure9am    130395 non-null  float64
 16  Pressure3pm    130432 non-null  float64
 17  Cloud9am       89572 non-null

- Date — дата, в которую зафиксировано наблюдение;
- Location — местонахождение метеорологической станции;
- MinTemp — минимальная температура (℃);
- MaxTemp — максимальная температура (℃);
- Rainfall — количество осадков (дождь) за сутки (мм);
- Evaporation — количество испарений до 9 утра (мм);
- Sunshine — количество часов в сутках, когда светило солнце;
- WindGustDir — направление самого сильного порыва ветра за последние 24 часа;
- WindGustSpeed — скорость самого сильного порыва ветра за последние 24 часа;
- WindDir9am — направление ветра в 9 утра;
- WindDir3pm — направление ветра в 3 часа дня;
- WindSpeed9am — скорость ветра в 9 часов утра;
- WindSpeed3pm — скорость ветра в 3 часа дня;
- Humidity9am — влажность в 9 утра;
- Humidity3pm — влажность в 3 часа дня;
- Pressure9am — атмосферное давление в 9 утра;
- Pressure3pm — атмосферное давление в 3 часа дня;
- Cloud9am — часть неба, закрытая облаками, в 9 утра;
- Cloud3pm — часть неба, закрытая облаками, в 3 часа дня;
- Temp9am — температура в 9 утра;
- Temp3pm — температура в 3 часа дня;
- RainToday — наличие дождя в этот день;
- RainTomorrow — наличие дождя на следующий день.

In [ ]:
# Выведем сколько суммарно пропусков в датафрейме.
print('Количество пропусков в датафрейме -', df.isnull().sum().sum())

Количество пропусков в датафрейме - 343248


In [ ]:
# Составим список с признаками, где пропусков меньше 40%.
ls_futures = []
ls = df.isnull().mean() * 100
for future in list(ls.index):
  if ls.loc[future] < 40:
    ls_futures.append(future)
print(ls_futures)
print('Количество признако, где пропусков больше 40% -', len(ls) - len(ls_futures))

['Date', 'Location', 'MinTemp', 'MaxTemp', 'Rainfall', 'WindGustDir', 'WindGustSpeed', 'WindDir9am', 'WindDir3pm', 'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm', 'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Temp9am', 'Temp3pm', 'RainToday', 'RainTomorrow']
Количество признако, где пропусков больше 40% - 3


In [ ]:
# Оставим в датафрейме только те признаки, где пропусков меньше 40%.
df = df[ls_futures]
# Выведем информацию.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145460 entries, 0 to 145459
Data columns (total 20 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           145460 non-null  object 
 1   Location       145460 non-null  object 
 2   MinTemp        143975 non-null  float64
 3   MaxTemp        144199 non-null  float64
 4   Rainfall       142199 non-null  float64
 5   WindGustDir    135134 non-null  object 
 6   WindGustSpeed  135197 non-null  float64
 7   WindDir9am     134894 non-null  object 
 8   WindDir3pm     141232 non-null  object 
 9   WindSpeed9am   143693 non-null  float64
 10  WindSpeed3pm   142398 non-null  float64
 11  Humidity9am    142806 non-null  float64
 12  Humidity3pm    140953 non-null  float64
 13  Pressure9am    130395 non-null  float64
 14  Pressure3pm    130432 non-null  float64
 15  Cloud9am       89572 non-null   float64
 16  Temp9am        143693 non-null  float64
 17  Temp3pm        141851 non-nul

In [ ]:
# Заменим Yes на 1.
df['RainToday'] = df['RainToday'].apply(lambda x: 1 if x == 'Yes' else x)
df['RainTomorrow'] = df['RainTomorrow'].apply(lambda x: 1 if x == 'Yes' else x)
# Заменим No на 0.
df['RainToday'] = df['RainToday'].apply(lambda x: 0 if x == 'No' else x)
df['RainTomorrow'] = df['RainTomorrow'].apply(lambda x: 0 if x == 'No' else x)
print(round(df['RainTomorrow'].mean(), 2))

0.22


In [ ]:
# Создадим признак с номерами месяцев.
df['Month'] = pd.to_datetime(df['Date']).dt.month
# Удалим признак Date.
df = df.drop('Date', axis=1)
# Выведем для просмотра датафрейм.
display(df.head())

,Location,MinTemp,MaxTemp,Rainfall,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Temp9am,Temp3pm,RainToday,RainTomorrow,Month
0,Albury,13.4,22.9,0.6,W,44.0,W,WNW,20.0,24.0,71.0,22.0,1007.7,1007.1,8.0,16.9,21.8,0.0,0.0,12
1,Albury,7.4,25.1,0.0,WNW,44.0,NNW,WSW,4.0,22.0,44.0,25.0,1010.6,1007.8,NaN,17.2,24.3,0.0,0.0,12
2,Albury,12.9,25.7,0.0,WSW,46.0,W,WSW,19.0,26.0,38.0,30.0,1007.6,1008.7,NaN,21.0,23.2,0.0,0.0,12
3,Albury,9.2,28.0,0.0,NE,24.0,SE,E,11.0,9.0,45.0,16.0,1017.6,1012.8,NaN,18.1,26.5,0.0,0.0,12
4,Albury,17.5,32.3,1.0,W,41.0,ENE,NW,7.0,20.0,82.0,33.0,1010.8,1006.0,7.0,17.8,29.7,0.0,0.0,12


In [ ]:
df_season = df.groupby('Month')['RainToday'].mean().sort_values(ascending=False)
print('Номер месяца в который выпадает больше всего осадков -', df_season.index[0])

Номер месяца в который выпадает больше всего осадков - 7


In [ ]:
# Составим список категориальных признаков.
categoricals = ['Month', 'Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm']
# Закодиуем данные.
df = pd.get_dummies(df, columns=categoricals)
# Выведем размер таблицы.
df.shape

(145460, 124)

In [ ]:
# Удалим все строки, где есть пропуски.
df = df.dropna(axis=0)
# Проверим результат.
print('Количесво пропусков в датафрейме -', df.isnull().sum().sum())

Количесво пропусков в датафрейме - 0


In [ ]:
# Создадим матрицу наблюдений.
X = df.drop('RainTomorrow', axis=1)
# Создадим вектор правильных ответов.
y = df['RainTomorrow']

In [ ]:
# Разобьем данные на тренировочную и тестовую выборки.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=31)

In [ ]:
# Иницианализируем модель логистической регрессии.
ln_model = linear_model.LogisticRegression()
# Обучим модель.
ln_model.fit(X_train, y_train)
# Сделаем предсказание.
y_pred_test = ln_model.predict(X_test)
# Выведем метрику.
print('roc_auc {:.2f}'.format(metrics.roc_auc_score(y_test, y_pred_test)))

roc_auc 0.73


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### Кросс-валидация

In [ ]:
# Зададим сетку параметров.
params = {
    'max_leaf_nodes': list(range(2, 10)), 
    'min_samples_split': [2, 3, 4], 
    'max_depth': [5,7,9,11]
}

# Создадим модель решающего дерева.
tr_model = tree.DecisionTreeClassifier(random_state=42)

# Вызываем класс GridSearchCV и передаем в него модель.
cv = GridSearchCV(
    estimator=tr_model, 
    param_grid=params, 
    cv=3,
    n_jobs=-1
)

# Обучим валидатор.
cv.fit(X_train, y_train)

# Выведем лучшие значения.
print(cv.best_params_)

{'max_depth': 5, 'max_leaf_nodes': 9, 'min_samples_split': 2}


In [ ]:
# Создадим модель решающего дерева с лучшими гиперпараметрами.
tr_model = tree.DecisionTreeClassifier(
    max_depth=5, 
    max_leaf_nodes=9,
    min_samples_split=2, 
    random_state=42
)
# Обучим модель.
tr_model.fit(X_train, y_train)
# Сделаем предсказание.
y_pred_test = tr_model.predict(X_test)
# Выведем метрику.
print('roc_auc {:.2f}'.format(metrics.roc_auc_score(y_test, y_pred_test)))

roc_auc 0.70


In [ ]:
# Построим модель случайного леса.
rf = ensemble.RandomForestClassifier(n_estimators=100, random_state=31)
# Обучим модель.
rf.fit(X_train, y_train)
# Сделаем предсказание.
y_pred_test = rf.predict(X_test)
# Выводим метрику.
print('roc_auc {:.2f}'.format(metrics.roc_auc_score(y_test, y_pred_test)))

roc_auc 0.73


In [ ]:
# Зададим параметры.
params = {'max_features': [ 4, 5, 6, 7], 'min_samples_leaf': [3, 5, 7, 9, 11], 'max_depth': [5, 10, 15]}

# Вызываем класс GridSearchCV и передаем в него модель.
cv_rf = GridSearchCV(
    estimator=rf,
    param_grid=params,
    cv=3,
    n_jobs=-1
)
# Обучаем модель.
cv_rf.fit(X_train, y_train)
# Выводим лучшие параметры.
print(cv_rf.best_params_)

{'max_depth': 15, 'max_features': 7, 'min_samples_leaf': 3}


In [ ]:
# Построим модель случайного леса.
rf = ensemble.RandomForestClassifier(
    n_estimators=100, 
    max_depth=15,
    max_features=7,
    min_samples_leaf=3,
    random_state=31
)
# Обучим модель.
rf.fit(X_train, y_train)
# Сделаем предсказание.
y_pred_test = rf.predict(X_test)
# Выводим метрику.
print('roc_auc {:.2f}'.format(metrics.roc_auc_score(y_test, y_pred_test)))

roc_auc 0.70


In [ ]:
# Выведем какие признаки оказывают большее влияние на
# целевую переменную по сравнению с другими.
feature_names = [x for x in df if x != 'RainTomorrow']
pd.DataFrame({'feat': feature_names,
              'coef': rf.feature_importances_}).sort_values(by='coef', ascending=False)

,feat,coef
7,Humidity3pm,0.250783
2,Rainfall,0.079757
6,Humidity9am,0.070403
10,Cloud9am,0.067092
9,Pressure3pm,0.065272
...,...,...
50,Location_Newcastle,0.000000
62,Location_SalmonGums,0.000000
51,Location_Nhil,0.000000
52,Location_NorahHead,0.000000
